In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

f = fits.open('/home/regier/bliss/data/sdss/2583/2/136/frame-r-002583-2-0136.fits')
w = WCS(f[0].header)

# lower-left corner of the 100x100-pixel study area is at pixel (310, 630)
w.pixel_to_world(310, 630)

In [ ]:
import pandas as pd

# catalog from https://catalogs.mast.stsci.edu/hsc/
# target: 323.3357504,-0.807026
# radius of 32 arcsec; v3; detailed; MagAuto <= 24
fn = '/home/regier/bliss/case_studies/dependent_tiling/HSC-10_20_2023.csv'
hst_catalog = pd.read_csv(fn)
hst_catalog

In [ ]:
from astropy.coordinates import SkyCoord

hst_coords = SkyCoord(ra=hst_catalog['MatchRA'], dec=hst_catalog['MatchDec'], unit='deg')
hst_pix = w.world_to_pixel(hst_coords)

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(f[0].data, origin='lower', cmap='gray_r')

In [ ]:
from matplotlib.patches import Rectangle

plt.imshow(f[0].data, origin='lower', cmap='gray_r')
plt.scatter(hst_pix[0], hst_pix[1], s=10, c='r')
rect = Rectangle((310, 630), 100, 100, linewidth=1, edgecolor='b', facecolor='none')
plt.gca().add_patch(rect)


In [ ]:
in_bounds = (hst_pix[0] > 310) & (hst_pix[0] < 410) & (hst_pix[1] > 630) & (hst_pix[1] < 730)
hst_inbounds = hst_catalog[in_bounds]

In [ ]:
plt.imshow(f[0].data, origin='lower', cmap='gray_r')
plt.scatter(hst_pix[0][in_bounds], hst_pix[1][in_bounds], s=10, c='r')
rect = Rectangle((310, 630), 100, 100, linewidth=1, edgecolor='b', facecolor='none')
plt.gca().add_patch(rect)

In [ ]:
hst_inbounds["MatchID"].unique().shape

In [ ]:
hst_inbounds

In [ ]:
import numpy as np
np.median(f[0].data)

In [ ]:
from bliss.surveys.sdss import SloanDigitalSkySurvey

psf_config = {"pixel_scale": 0.396, "psf_slen": 25}
rcf_conf = [{"run": 2583, "camcol": 2, "fields": [136,]}]

sdss_obj = SloanDigitalSkySurvey(
    psf_config,
    rcf_conf,
    dir_path="/data/sdss",
    load_image_data = True,
)

img = sdss_obj.items[0]["image"][2][630:730, 310:410]
bkgd = sdss_obj.items[0]["background"][2][630:730, 310:410]

In [ ]:
np.median(img - bkgd)

In [ ]:
np.mean(img - bkgd)

In [ ]:
from os import environ
environ["CUDA_VISIBLE_DEVICES"] = "1"

from pathlib import Path
import torch
from hydra import initialize, compose
from hydra.utils import instantiate
from bliss.encoder import Encoder


environ["BLISS_HOME"] = str(Path().resolve().parents[1])
with initialize(config_path="../../case_studies/dependent_tiling/", version_base=None):
    cfg = compose("config", {"encoder.two_layers=true"})

encoder: Encoder = instantiate(cfg.encoder)
encoder.load_state_dict(torch.load("../../case_studies/dependent_tiling/m2.pt"))
encoder.eval()
encoder.cuda()

torch.set_grad_enabled(False)

In [ ]:
#batch = {
#    "images": torch.from_numpy(img).unsqueeze(0).cuda(),
#    "background": torch.from_numpy(bkgd).unsqueeze(0).cuda(),
#}
#encoder.predict_step(batch, None, None)

In [18]:
from bliss.predict import predict_dataset

predict_dataset(cfg)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/regier/bliss/.venv/lib/python3.10/site-packages/hydra/_internal/instantiate/_instantiate2.py", line 92, in _call_target
    return _target_(*args, **kwargs)
  File "/home/regier/bliss/bliss/surveys/decals.py", line 134, in __init__
    self.psf = DECaLS_PSF(
  File "/home/regier/bliss/bliss/surveys/decals.py", line 532, in __init__
    self.psf_galsim[brickname] = self._get_psf_coadded(
  File "/home/regier/bliss/bliss/surveys/decals.py", line 551, in _get_psf_coadded
    des_obj, idx = DECaLS.constituent_obj(
  File "/home/regier/bliss/bliss/surveys/decals.py", line 98, in constituent_obj
    idx = cls.constituent_objs.idx(des_image_id)
  File "/home/regier/bliss/bliss/surveys/des.py", line 122, in idx
    return self.image_id_list.index(self.to_dictconfig(image_id))
ValueError: {'sky_coord': {'ra': 337.05042330944565, 'dec': -0.8148310474424231}, 'decals_brickname': '3368m010', 'ccdname': 'S15', 'g': 'decam/CP/V4.8.2a/CP20141020/c4d_14